In [7]:
from rdflib import Graph
g = Graph()
g.parse("../../resources/eurovoc-skos-ap-eu.rdf")

<Graph identifier=Ne22f6ec57c1f4a668e8dc476dd1cd781 (<class 'rdflib.graph.Graph'>)>

In [8]:
g

<Graph identifier=Ne22f6ec57c1f4a668e8dc476dd1cd781 (<class 'rdflib.graph.Graph'>)>

In [9]:
for i, (s, p, o) in enumerate(g):
    print((s, p, o))
    if i == 10:
        break

(rdflib.term.URIRef('http://eurovoc.europa.eu/xl_fi_5faf1954'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#Label'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/844'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), rdflib.term.Literal('еколошки дисбаланс', lang='mk'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/2225'), rdflib.term.URIRef('http://www.w3.org/2008/05/skos-xl#prefLabel'), rdflib.term.URIRef('http://eurovoc.europa.eu/466502'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/175161'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#versionInfo'), rdflib.term.Literal('n/a'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/140616'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#versionInfo'), rdflib.term.Literal('n/a'))
(rdflib.term.URIRef('http://eurovoc.europa.eu/6755'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), rdflib.term.Literal('start-up',

In [11]:
from rdflib import SKOS, Graph, Literal, URIRef
from unidecode import unidecode
import logging


# Function to search concepts by literal string
def search_concepts_fast(graph, literal_string):
    results = []
    # Search for the literal in the graph
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'), Literal(literal_string, lang='nl'))):
        results.append(s)
    return results

def search_concepts_slow(graph, literal_string):
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and unidecode(o.lower()) == unidecode(literal_string.lower()):
            results.append(s)
    
    if results:
        return results
    
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and unidecode(o.lower()) == unidecode(literal_string.lower()):
            results.append(s)
    
    if results:
        return results
    
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#prefLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and "".join(unidecode(literal_string.lower()).split()) in "".join(unidecode(o.lower()).split()):
            results.append(s)
    
    if results:
        return results
    
    results = []
    # Iterate over all triples with the prefLabel property
    for s, p, o in graph.triples((None, URIRef('http://www.w3.org/2004/02/skos/core#altLabel'), None)):
        # Check if the object's value matches the literal_string, ignoring case
        if o.language == 'nl' and "".join(unidecode(literal_string.lower()).split()) in "".join(unidecode(o.lower()).split()):
            results.append(s)
    
    if results:
        return results
    
    return []
    


# Function to retrieve broader concepts
def find_broader_concepts(graph, concept_uri):
    broader_concepts = []
    for s, p, o in graph.triples((concept_uri, URIRef('http://www.w3.org/2004/02/skos/core#broader'), None)):
        broader_concepts.append((s, p, o))
        broader_concepts.extend(find_broader_concepts(graph, o))  # Recursively find broader concepts
    return broader_concepts

# Function to find Top Concept Of the broadest concept
def find_top_concept_of(graph, concept_uri):
    top_concept_of = []
    for s, p, o in graph.triples((concept_uri, URIRef('http://www.w3.org/2004/02/skos/core#topConceptOf'), None)):
        top_concept_of.append((s, p, o))
        top_concept_of.extend(find_top_concept_of(graph, o))  # Recursively find broader concepts
    return top_concept_of

# Function to find Domain of a Top Concept
def find_domain(graph, top_concept_uri):
    domain = []
    for s, p, o in graph.triples((top_concept_uri, URIRef('http://publications.europa.eu/ontology/euvoc#domain'), None)):
        domain.append((s, p, o))
        domain.extend(find_domain(graph, o))  # Recursively find broader concepts
    return domain

# Function to search concepts by literal string
def search_literal(graph, domain_uri):
    results = []
    # Search for the prefLabel property in the given domain_uri specifically in Dutch
    for s, p, o in graph.triples((domain_uri, SKOS.prefLabel, None)):
        if o.language == 'nl':
            results.append((s, p, str(o)))  # Convert Literal to string for better readability
    return results


def get_domain(query):
    # Example usage
    concept_search = query.lower()
    concepts = search_concepts_fast(g, concept_search)
    if not concepts:
        concepts = search_concepts_slow(g, concept_search)
    # print("Found concepts:", concepts)
    # print(search_literal(g, concepts[0])[0][2])
    # print("\n")

    # Concepts error handling
    if len(concepts) == 0:
        raise ValueError("Concept not found")
    elif len(concepts) > 1:
        logging.warning(f"More than 1 concept found? {[search_literal(g, c)[0][2] for c in concepts]}\nPicking first one.")

    concept = concepts[0]

    # Get broader concepts for a found concept
    broader = find_broader_concepts(g, concept)
    # print("Broader concepts:", broader)

    if broader:
        # print(search_literal(g, broader[-1][2])[0][2])
        concept = broader[-1][2]

    # print("\n")

    # Get the top concept of the broadest concept
    top = find_top_concept_of(g, concept)
    # print("Top concepts:", top)
    # print(search_literal(g, top[-1][2])[0][2])

    # print("\n")

    # Get the top concept of the broadest concept
    domain = find_domain(g, top[-1][2])
    # print("Domain:", domain)
    # print(search_literal(g, domain[0][2])[0][2])

    return search_literal(g, domain[0][2])[0][2]

get_domain("COMMUNAUTAIRE HULP")

'16 ECONOMIE'

In [12]:
import json
import os

folder_path = "../../src/parlementaire_stukken/json"

kamerstukken = []
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        kamerstukken.append((filename, data))
print(len(kamerstukken))

4015


In [13]:
import json

from common.text_corrections import fix_name

with open("../../members.json", "r") as fp:
    name_metadata = json.load(fp)
name_to_party = {fix_name(n["name"], swap_first_last_name=True): n["party"] for n in name_metadata}

In [15]:
# Pseudocode
from collections import defaultdict
from tqdm import tqdm
from common.text_corrections import fix_name


parties_to_domains = defaultdict(lambda: defaultdict(int))
for stukname, kamerstuk in tqdm(kamerstukken):
    try:
        type_of_document = kamerstuk["Kamer en/of Senaat"]["Hoofddocument"][-1]["Type"][-1]["NL"]
        if type_of_document.lower() == "verslag":
            continue

        # Descriptors (either a hoofddescriptor or one of the subdescriptors)
        if "Eurovoc-hoofddescriptor" in kamerstuk["Descriptoren, trefwoorden"]:
            hoofddescriptor = kamerstuk["Descriptoren, trefwoorden"]["Eurovoc-hoofddescriptor"]["NL"]
        else:
            hoofddescriptor = kamerstuk["Descriptoren, trefwoorden"]["Eurovoc descriptoren"][0]["NL"]

        # Authors
        auteurs = kamerstuk["Kamer en/of Senaat"]["Hoofddocument"][-1]["Auteur"]

        try:
            domain = get_domain(hoofddescriptor)
        except ValueError:
            domain = hoofddescriptor
        parties_seen = set()
        for auteur in auteurs:
            if auteur["Partij"] == "ZZZ":
                parties_to_domains["ZZZ"][domain] += 1
            else:
                name = fix_name(f"{auteur['Naam']} {auteur['Voornaam']}", swap_first_last_name=False)
                if name in name_to_party:
                    party = name_to_party[name]
                else:
                    party = auteur["Partij"]
                if party not in parties_seen:
                    parties_to_domains[party][domain] += 1
                    parties_seen.add(party)
    except KeyError:
        print(stukname)


  3%|▎         | 111/4015 [00:10<05:21, 12.13it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
  3%|▎         | 133/4015 [00:13<06:46,  9.55it/s]WARNING:root:More than 1 concept found? ['overheidsopdrachten', 'Overeenkomst inzake overheidsopdrachten', 'overheidsopdracht voor leveringen', 'overheidsopdracht voor werken', 'groene overheidsopdracht']
Picking first one.
  6%|▌         | 249/4015 [00:27<08:27,  7.42it/s]WARNING:root:More than 1 concept found? ['personeelsbenoeming', 'benoeming van de leden', 'benoemingsbevoegdheid']
Picking first one.
  6%|▋         | 256/4015 [00:29<09:54,  6.32it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verst

55K1445.json


 11%|█         | 431/4015 [00:46<03:59, 14.96it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
 21%|██        | 838/4015 [01:09<03:05, 17.09it/s]

55K0811.json


 24%|██▎       | 946/4015 [01:18<06:29,  7.87it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
 25%|██▌       | 1008/4015 [01:25<05:46,  8.68it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
 26%|██▌       | 1046/4015 [01:29<04:33, 10.84it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon 

...

 81%|████████▏ | 3263/4015 [04:13<00:54, 13.68it/s]WARNING:root:More than 1 concept found? ['overheidsopdrachten', 'Overeenkomst inzake overheidsopdrachten', 'overheidsopdracht voor leveringen', 'overheidsopdracht voor werken', 'groene overheidsopdracht']
Picking first one.
 82%|████████▏ | 3312/4015 [04:17<00:42, 16.68it/s]

55K1443.json


 83%|████████▎ | 3332/4015 [04:19<01:03, 10.77it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
 86%|████████▌ | 3438/4015 [04:27<00:33, 17.02it/s]

55K1610.json


 90%|█████████ | 3616/4015 [04:40<00:42,  9.33it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
 91%|█████████ | 3636/4015 [04:45<01:27,  4.31it/s]WARNING:root:More than 1 concept found? ['persoon met een lichamelijke beperking', 'persoon met een verstandelijke beperking', 'persoon met een verstandelijke beperking', 'faciliteiten voor mensen met een beperking', 'werknemer met een beperking', 'buitengewoon onderwijs']
Picking first one.
 91%|█████████ | 3637/4015 [04:47<02:18,  2.74it/s]WARNING:root:More than 1 concept found? ['personeelsbenoeming', 'benoeming van de leden', 'benoemingsbevoegdheid']
Picking first one.
 92%|█████████▏| 3709/4015 [04:54<00:25, 11.85it/s]WARNING:root:More than 1 concept found? ['personeelsbenoeming', 'benoeming v

In [17]:
parties_to_domains

defaultdict(<function __main__.<lambda>()>,
            {'ZZZ': defaultdict(int,
                         {'08 INTERNATIONALE BETREKKINGEN': 97,
                          '66 ENERGIE': 50,
                          '28 SOCIALE VRAAGSTUKKEN': 156,
                          '32 OPVOEDING, ONDERWIJS EN COMMUNICATIE': 30,
                          '04 POLITIEK': 73,
                          '12 RECHT': 110,
                          '24 FINANCIËN': 144,
                          '44 WERKGELEGENHEID EN ARBEID': 50,
                          '48 TRANSPORT': 20,
                          '40 ONDERNEMING EN CONCURRENTIE': 11,
                          'GEESTELIJK GEHANDICAPTE': 1,
                          '16 ECONOMIE': 17,
                          '20 ECONOMIE, VERKEER EN HANDELSVERKEER': 18,
                          '68 INDUSTRIE': 8,
                          'PROGRAMMAWET': 5,
                          '76 INTERNATIONALE ORGANISATIES': 9,
                          '36 WETENSCHAPPEN': 3

In [19]:
import plotly.graph_objects as go

# Setup data for absolute amounts
def plot_absolute(data):
    fig = go.Figure()
    for party, subjects in data.items():
        fig.add_trace(go.Bar(
            name=party,
            x=list(subjects.keys()),
            y=list(subjects.values()),
        ))
    fig.update_layout(
        title="Absolute Amounts per Subject by Party",
        xaxis_title="Subject",
        yaxis_title="Amount",
        barmode='group',
        xaxis={'categoryorder':'total descending'},
        height=1080
    )
    fig.show()

plot_absolute(parties_to_domains)

In [22]:
colors = {
    "DéFI": "#FFA500",       # Orange
    "N-VA": "#FFD700",       # Yellow
    "Les Engagés": "#FF8C00",# Darker Orange
    "VB": "#000000",         # Black
    "PVDA": "#FF0000",       # Bright Red
    "Open Vld": "#1E90FF",   # Dodger Blue
    "PTB": "#CD5C5C",        # Indian Red
    "MR": "#87CEEB",         # Sky Blue
    "Vooruit": "#DC143C",    # Crimson
    "Ecolo": "#228B22",      # Forest Green
    "Groen": "#32CD32",      # Lime Green
    "CD&V": "#FF7F00",       # Deep Orange
    "PS": "#B22222",         # Firebrick Red
    "cd&v": "#FFF",          # White
    "ZZZ": "#1D1D1D"         # Grey
}


def calculate_relative(data):
    relative_data = {}
    for party, subjects in data.items():
        total = sum(subjects.values())
        relative_data[party] = {subject: value / total for subject, value in subjects.items()}
    return relative_data

def plot_relative(data):
    relative_data = calculate_relative(data)
    fig = go.Figure()
    for party, subjects in relative_data.items():
        fig.add_trace(go.Bar(
            name=party,
            x=list(subjects.keys()),
            y=list(subjects.values()),
            marker_color=colors[party]
        ))
    fig.update_layout(
        title="Amount of legislation documents with at least one author of a party, split by EuroVoc Domain, relative to the amount of seats for that party",
        xaxis_title="Subject",
        yaxis_title="Percentage",
        barmode='group',
        xaxis={'categoryorder':'total descending'},
        height=1080,
        width=1920
    )
    fig.show()

plot_relative(parties_to_domains)